# Stratification on propensity score study

Creating some sample calculation for stratified ATT calculation on propensity score.

In [1]:
import numpy as np
import statsmodels.api as sm

from scipy.stats import norm, bernoulli

In [2]:
# generating data
s = 10000
# covariate 1
x1 = bernoulli.rvs(p=.1, size=s)
x2 = bernoulli.rvs(p=.1, size=s)

# treatment is caused by x1 only
p1 = .9
p2 = .1
d = bernoulli.rvs(p=p1, size=s) * x1 + bernoulli.rvs(p=p2, size=s) * (1 - x1)

# some dummy caused by treatment but not causing y
x3 = bernoulli.rvs(p=.8, size=s) * d + bernoulli.rvs(p=.2, size=s) * (1 - d)

# outcome is caused by x1, x2 and treatment
# ATE is 2
y = x1 + x2 + 2 * d

# naive ATE estimation based on observed data is biased
(y * d).sum() / d.sum() - (y * (1 - d)).sum() / (1 - d).sum()

np.float64(2.508738481497092)

In [3]:
# calculate propensity score

# Design matrix with intercept
X = np.column_stack([x1, x2])
X = sm.add_constant(X, has_constant='add')

# Fit logistic regression
model = sm.Logit(d, X).fit()

print(model.summary())


Optimization terminated successfully.
         Current function value: 0.324758
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                10000
Model:                          Logit   Df Residuals:                     9997
Method:                           MLE   Df Model:                            2
Date:                Sat, 22 Nov 2025   Pseudo R-squ.:                  0.3174
Time:                        19:45:42   Log-Likelihood:                -3247.6
converged:                       True   LL-Null:                       -4757.6
Covariance Type:            nonrobust   LLR p-value:                     0.000
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -2.2167      0.037    -59.654      0.000      -2.290      -2.144
x1             4.4682      0.

In [4]:
# check predictions
X_new = np.array([
    [1, 0],
    [1, 1],
    [0, 0],
    [0, 1]
])
X_new = sm.add_constant(X_new, has_constant='add')  # <-- force constant

model.predict(X_new)
# -> propensity scores are roughly determined by x1
# Note the true weights are 0.9 and 0.1, so we are close

array([0.90478197, 0.92505366, 0.09826121, 0.12399387])

In [5]:
# calcualte ATE based on strata of same propensity score (given by strata of x1)

# mean of treated units with propensity ~ 0.9
m1 = (y * x1 * d).sum() / (x1 * d).sum()
# mean of untreatet units with propensity ~ 0.9
m2 = (y * x1 * (1 - d)).sum() / (x1 * (1 - d)).sum()

# mean of treated units with propensity ~ 0.2
m3 = (y * (1 - x1) * d).sum() / ((1 - x1) * d).sum()
# mean of untreatet units with propensity ~ 0.2
m4 = (y * (1 - x1) * (1 - d)).sum() / ((1 - x1) * (1 - d)).sum()

m1 - m2, m3 - m4
# -> both are close to the real ATE of 2,
# so the estimate on the full population will also be close to 2

(np.float64(2.01535657325131), np.float64(2.025593053833882))